In [5]:
"""For altering memory usage"""
from sage.interfaces.gap import get_gap_memory_pool_size, set_gap_memory_pool_size
n = get_gap_memory_pool_size()
set_gap_memory_pool_size(200*n)
n = get_gap_memory_pool_size()
import time
import multiprocessing 
from multiprocessing import Process, Pool
print("Number of cpu : ", multiprocessing.cpu_count())

Number of cpu :  8


In [2]:
"""List of indices of small groups by GAP library: [x,y]."""
small = []
for i in range(299,300):
    if i not in [639,671]:
        for groupid in gap.IdsOfAllSmallGroups(i+1):
            G = gap.SmallGroup(groupid)
            G = gap.Image(gap.IsomorphismPermGroup(G))
            G = PermutationGroup(gap_group = G)
            small.append(G)
#         print(i+1)
    else:
        print(i+1)
print(len(small))

49


In [6]:
"""For new problem with general subgroups."""
class induce():
    """Function to return list of class sizes in default order."""
    def __init__(self):
        pass

    """Returns class sizes up to conjugacy"""
    def class_sizes(self, G):
        group_order = G.order()
        reps = G.conjugacy_classes_representatives()
        class_sizes = []
        for g in reps:
            class_sizes.append(group_order / G.centralizer(g).order())
#         print(class_sizes)
#         orders = []
#         for rep in reps:
#             orders.append(rep.order())
#         print(orders)
        return class_sizes

    """Function to find all subgroups. Returns list of subgroups."""
    def subgroups(self, G):
        subgroups = G.conjugacy_classes_subgroups()
#         print(subgroups)
        return subgroups

    """Function to find induced trivial character of each subgroup. Returns a list of lists."""
    def get_induced(self, subgroups, G):
        ind = []
        for subgroup in subgroups:
            ind.append(subgroup.trivial_character().induct(G))
#         print(ind)
        return ind

    """Forms the matrices that we use to solve the simultaneous equation later. The last entry is the trivial representation."""
    def mat(self, induced):
        matrix = []
        for char in induced:
            matrix.append(list(char.values()))
        neg = []
        for i in range(len(matrix[0])):
            neg.append(-1)
        matrix[-1] = neg
#         for line in matrix:
#             print(line)
        return matrix

    """Check if a subgroup H of G is core-free"""
    def core_free(self, G, H):
        if H.order() == 1: 
            return True
        core = self.subgroups(H)[1:]
        for subsub in core:
            if subsub.is_normal(G) == True:
                return False
        return True

    """Find all subgroups of G containing H (just the conjugates)"""
    def subgroups_containing(self, G, H, subgroups):
        subgroups_list = [] 
        h = H.order()
        H_conjugates = []
        H_normalizer = G.normalizer(H)
        H_norm_cosets = G.cosets(H_normalizer)
        for coset in H_norm_cosets:
            g = coset[0]
            H_conjugates.append(H.conjugate(g))
        for subgroup in subgroups:
            if subgroup.order() > h:
                sub = False
                i = len(H_conjugates)
                while sub == False and i != 0:
                    sub = H_conjugates[i-1].is_subgroup(subgroup)
                    i -= 1
                if sub == True:
                    subgroups_list.append(subgroup)
        return subgroups_list
    
    """Finding PHG"""
    def get_phg(self, G, subgroup,j):
        gens = []
        gens += subgroup.list()
        X = []
        elems = G.list()
        for g in elems:
            for h in subgroup.conjugate(g).list():
                if h not in X:
                    X.append(h)
        for g in elems:
            if g**j not in X:
                gens.append(g)
        hp_group = PermutationGroup(gens)
        print("Hp" + str(gap.IdSmallGroup(hp_group)), str(gap.IdSmallGroup(subgroup)))
            

    """Test if there exists non-negative solutions. Prints the id of group that works and the ci, prints fail otherwise.
    phg option to test hypothesis
    """
    def test(self, G):
        """Filter abelian groups"""
        center = G.center()
        gOrder = G.order() 
        if center.order() == gOrder:
            j = 2
            for elem in center.list():
                j = max(j, elem.order())
                if is_prime(j) != True:
                    return ""
            if is_prime(gOrder) == True:
                return ""
            else:
                return "1,1  "

        """Begin of main test"""
        a = self.class_sizes(G)
        b = self.subgroups(G)
        zeros = [0]*len(a)
        ret_list = ""
        for i in range(len(b)):
            cor = self.core_free(G, b[i])
            if cor == True:
                try:
#                     """Finds P_H(G)"""
#                     self.get_phg(G,b[i],j)

                    b_i = self.subgroups_containing(G, b[i], b)
                    c = self.get_induced(b_i, G)
                    d_i = self.mat(c)
                    e = Matrix(d_i).T
                    temp = b[i].trivial_character().induct(G).values()
                    Y = vector(temp)
                    p = MixedIntegerLinearProgram(solver='ppl')
                    w = p.new_variable(nonnegative=True)
                    p.set_objective(0)
                    p.add_constraint(e*w == Y)
                    p.solve()
                    
                    """Finds the decomposition structure."""
                    print("Induced char of H: " + str(temp))
                    out = []
                    print("Subgroups in decomposition")
                    for j, v in (p.get_values(w).items()):
                        out.append('w_%s = %s' % (j, v))
                        """Prints subgroup and subgroup order."""
                        if round(v,6) != 0:
                            print(e.T[j])
                            print("Coef:" + str(v), "Order:" + str(gap.IdSmallGroup(b_i[j])))
#                     print(e.T)
#                     print(out)

                    """Prints the solution and SmallGroup Id"""
                    ret_list += str(gap.IdSmallGroup(b[i])[1]) + "," + str(gap.IdSmallGroup(b[i])[2])+"  "
                    print("Pair G-H:" + str(gap.IdSmallGroup(G)[1]) + "," + str(gap.IdSmallGroup(G)[2]) +"    "+ str(gap.IdSmallGroup(b[i])[1]) + "," + str(gap.IdSmallGroup(b[i])[2]))
                except:
                    pass
#                     print('fail')
        return ret_list

In [7]:
"""Main function
If group is in id form.
https://www.gap-system.org/Manuals/pkg/SmallGrp-1.3/doc/chap1.html
This works on CoCalc/non-windows system with gap_packages library installed."""
if __name__ == "__main__":
    inducer = induce()
    groups = [[720,765],[900,90],[1029,11],[1440,5844],[1620,419],[1620,420],[1764,140],[1800,565]]
    out_a = []
    out_b = []
    start = time.time()
    for groupid in groups:
        G = gap.SmallGroup(groupid)
        G = gap.Image(gap.IsomorphismPermGroup(G))
        ret = inducer.test(PermutationGroup(gap_group = G))
        if len(ret) > 0:
            out_a.append(groupid)
            out_b.append(ret)
    end = time.time()
    print(end-start)
#         try:
#             inducer.test(PermutationGroup(gap_group = G.AsPermGroup()))
#         except:
#             pass
    for a in out_a:
        print(str(a[1])+","+str(a[2]))
    for b in out_b:
        print(b)
    """If group is in SAGE form.
    https://doc.sagemath.org/html/en/thematic_tutorials/group_theory.html#groups-of-small-order-as-permutation-groups"""
#     C5 = CyclicPermutationGroup(5)
#     G = C5.holomorph()
#     C2 = CyclicPermutationGroup(2)
#     groups = [direct_product_permgroups([C2,G])]
#     for group in groups:
#         try:
#             result = inducer.test(group)
#         except:
#             pass

Induced char of H: [180, 4, 0, 4, 0, 0, 0, 0]
Subgroups in decomposition
(90, 2, 0, 2, 0, 2, 2, 0)
Coef:1 Order:[ 8, 1 ]
(90, 2, 2, 2, 0, 0, 0, 0)
Coef:1 Order:[ 8, 4 ]
(2, 2, 0, 2, 2, 0, 0, 2)
Coef:1 Order:[ 360, 118 ]
(-1, -1, -1, -1, -1, -1, -1, -1)
Coef:2 Order:[ 720, 765 ]
Pair G-H:720,765    4,1
Induced char of H: [90, 2, 0, 10, 0, 0, 0, 0]
Subgroups in decomposition
(45, 1, 1, 5, 0, 1, 1, 0)
Coef:2 Order:[ 16, 8 ]
(2, 2, 0, 2, 2, 0, 0, 2)
Coef:1 Order:[ 360, 118 ]
(-1, -1, -1, -1, -1, -1, -1, -1)
Coef:2 Order:[ 720, 765 ]
Pair G-H:720,765    8,3
Induced char of H: [450, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Subgroups in decomposition
(225, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1)
Coef:3/2 Order:[ 4, 1 ]
(150, 0, 0, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
Coef:1/4 Order:[ 6, 2 ]
(150, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 0)
Coef:1/4 Order:[ 6, 2 ]
(150, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 0, 0, 0)
Coef:1/4 Order:[ 6, 2 ]
(6, 6, 6, 6, 

Induced char of H: [147, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21, 0, 0, 0, 0, 0, 0, 0, 21, 0, 0]
Subgroups in decomposition
(49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 7, 1, 1)
Coef:3 Order:[ 21, 1 ]
(3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0)
Coef:1 Order:[ 343, 3 ]
(-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1)
Coef:3 Order:[ 1029, 11 ]
Pair G-H:1029,11    7,1
Induced char of H: [147, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21, 0, 21, 0, 0, 0, 0, 0, 0, 0]
Subgroups in decomposition
(49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0, 0, 0, 1, 1)
Coef:3 Order:[ 21, 1 ]
(3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0)
Coef:1 Order:[ 343, 3 ]
(-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1)
Coef:3 Order:[ 1029, 11 ]
Pair G-H:1029,11    7,1
Induced char of H: [360, 0, 0, 8, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0]
Subgroups in decomposition
(180, 0, 0, 4, 4, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0)
Co

Coef:27/2 Order:[ 1620, 420 ]
Pair G-H:1620,420    1,1
Induced char of H: [810, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Subgroups in decomposition
(270, 0, 0, 27, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
Coef:1/4 Order:[ 6, 1 ]
(270, 0, 0, 0, 27, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
Coef:1/4 Order:[ 6, 1 ]
(270, 0, 27, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
Coef:1/4 Order:[ 6, 1 ]
(270, 27, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
Coef:1/4 Order:[ 6, 1 ]
(81, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
Coef:27/4 Order:[ 20, 2 ]
(-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1)
Coef:27/4 Order:[ 1620, 420 ]
Pair G-H:1620,420    2,1
Induced char of H: [405, 0, 0, 0, 0, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Subgroups in decomposition
(81, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

9544.938275814056


IndexError: list index out of range

In [4]:
G = gap.SmallGroup([1944,3946])
G = gap.Image(gap.IsomorphismPermGroup(G))
gap.StructureDescription(G)

(C3 x C3 x C3 x C3 x A4) : C2